Majority vote from team's CV

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import glob

from tqdm import tqdm
import seaborn as sns
from collections import Counter

import cv2

In [2]:
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors

In [3]:
from collections import Counter
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize

In [4]:
from collections import defaultdict

In [5]:
import bz2
import pickle
import _pickle as cPickle

def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2', 'w') as f:
        cPickle.dump(data, f)
        
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data

In [6]:
#exps = !ls ../input/whale-21-3-2022/ | grep pbz2 | grep fold4
#exps

In [7]:
FOLD=0

In [8]:
exps_all = [
    [
         'exp16v6Augmix_fold0.pbz2',
         'exp16v4sampler_fold0.pbz2',
         'exp16v6AugMixNew4_fold0.pbz2',
         'exp16v6AugMixNew4_fold0_tta.pbz2',
         'exp16v6AugmixB3_fold0.pbz2',
         'exp16v6AugmixB4Step2_fold0.pbz2',
         'exp16v6AugmixB4Step2_fold0_remove_background.pbz2',
         'exp16v6AugmixB4_fold0.pbz2',
         'exp16v6AugmixB4_fold0_remove_background.pbz2',
         'exp16v6AugmixFullB4_fold0.pbz2',
         'exp16v6sampler_fold0.pbz2',
         'exp21b6_fold0.pbz2',
         'exp21b6_fold0_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold1.pbz2',
         'exp16v6AugMixNew4_fold1.pbz2',
         'exp16v6AugMixNew4_fold1_tta.pbz2',
         'exp16v6AugmixB3_fold1.pbz2',
         'exp16v6AugmixB4Step2_fold1.pbz2',
         'exp16v6AugmixB4Step2_fold1_remove_background.pbz2',
         'exp16v6AugmixB4_fold1.pbz2',
         'exp21b6_fold1_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold2.pbz2',
         'exp16v6AugMixNew4_fold2.pbz2',
         'exp16v6AugMixNew4_fold2_tta.pbz2',
         'exp16v6AugmixB3_fold2.pbz2',
         'exp16v6AugmixB4Full_fold2.pbz2',
         'exp16v6AugmixB4Step2_fold2.pbz2',
         'exp16v6AugmixB4_fold2.pbz2',
         'exp16v6AugmixB4_fold2_remove_background.pbz2',
         'exp21b6_fold2_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold3.pbz2',
         'exp16v6AugMix1536_fold3.pbz2',
         'exp16v6AugmixB3_fold3.pbz2',
         'exp16v6AugmixB4Full_fold3.pbz2',
         'exp16v6AugmixB4Step2_fold3.pbz2',
         'exp16v6AugmixB4_fold3_remove_background.pbz2',
         'exp21b6_fold3_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold4.pbz2',
         'exp16v6AugMix1536_fold4.pbz2',
         'exp16v6AugmixB3_fold4.pbz2',
         'exp16v6AugmixB4Step2_fold4.pbz2',
         'exp16v6AugmixB4_fold4_remove_background.pbz2',
    ]
]

In [9]:
exps = exps_all[FOLD]

In [10]:
fold2 = pd.read_csv("/kaggle/happywhale/input/whale-21-3-2022/fold2.csv")
fold2.shape

(51033, 8)

In [11]:
valid = fold2.query("fold == @FOLD").reset_index()
train = fold2.query("fold != @FOLD").reset_index()

In [12]:
valid_ind = (fold2.fold == FOLD).values
train_ind = (fold2.fold != FOLD).values

In [13]:
train_inddividual_id_lis = train["individual_id"].values
valid_inddividual_id_lis = valid["individual_id"].values

def get_result_each(files):
    DIRNAME ="../input/whale-21-3-2022/"
      
    ret = []
        
    for fname in files:
        print(fname)
        data = decompress_pickle(DIRNAME + fname  )
        
        train_emb = data["emb_val"]#[new_index,:]
        
        train_emb=normalize(train_emb)
        
        valid_preds = train_emb[valid_ind]
        train_preds = train_emb[train_ind]
        
        
        distances = pairwise_distances(valid_preds,train_preds)
                
        K=1000
        predicted_positions = np.argpartition(distances, K, axis=1)[:, :K]

        all_pred_lis = []
        for i in tqdm(range(predicted_positions.shape[0])):
            nearest = [(train_inddividual_id_lis[j], distances[i, j]) for j in predicted_positions[i]]
            nearest.sort(key=lambda x: x[1])

            prediction = [index_id for index_id, d in nearest]

            done = set()
            pred_list = []
            for pred_id in prediction:
                if pred_id in done:
                    continue
                done.add(pred_id)
                pred_list.append(pred_id)
                if len(pred_list)==20:
                    break

            all_pred_lis.append(pred_list)
        
        ret.append(all_pred_lis)
        
    return ret
    

### get result for each model

In [14]:
ret = get_result_each(exps)

exp16v6Augmix_fold0.pbz2


100%|██████████| 8372/8372 [00:07<00:00, 1118.06it/s]


exp16v4sampler_fold0.pbz2


100%|██████████| 8372/8372 [00:07<00:00, 1130.66it/s]


exp16v6AugMixNew4_fold0.pbz2


...

100%|██████████| 8372/8372 [00:07<00:00, 1125.07it/s]


exp21b6_fold0.pbz2


100%|██████████| 8372/8372 [00:07<00:00, 1073.93it/s]


exp21b6_fold0_remove_background.pbz2


100%|██████████| 8372/8372 [00:07<00:00, 1123.12it/s]


In [15]:
len(ret),len(ret[0]),len(ret[0][0])

(13, 8372, 20)

In [16]:
def get_score(weights):
    
    #train_emb
    vote_result = defaultdict(lambda:defaultdict(int))
    
    for preds, w in zip(ret, weights):
        for i in range(len(preds)):
            
            for j,p in enumerate(preds[i]):
                vote_result[i][p] += (w / (j+1))
        
        
    score_lis = []
    pred_ids = []

    for i in range(len(ret[0])):
        pred_list = np.array(sorted(vote_result[i].items(), key=lambda item: item[1])[::-1])[:5,0].tolist()

        if valid_inddividual_id_lis[i] in pred_list:
            s = 1/(pred_list.index(valid_inddividual_id_lis[i]) + 1)
            score_lis.append(s)
        else:
            score_lis.append(0)
    
    return np.mean(score_lis)

In [17]:
# un-weighted result
get_score([1] * len(exps))

0.8566431756649148

optimize by optuna

In [18]:
import optuna

In [19]:
def objective_wrapper(exps):
    def objective(trial):

        weights = [1]
        for i in range(len(exps)-1):
            w=trial.suggest_uniform(exps[i+1], 0, 4)
            weights.append(w)
        
        
        score_lis = get_score(weights)
        score = np.mean(score_lis)
        return score
    
    return objective


In [20]:
%%time

study = optuna.create_study(direction='maximize')
study.optimize(objective_wrapper(exps), timeout=300)

[I 2022-04-17 07:47:35,619] A new study created in memory with name: no-name-d3e0a303-9e5c-443d-9213-4aaa6a007fcc
[I 2022-04-17 07:47:37,356] Trial 0 finished with value: 0.8521798853320592 and parameters: {'exp16v4sampler_fold0.pbz2': 1.096956826159703, 'exp16v6AugMixNew4_fold0.pbz2': 1.4615008605815847, 'exp16v6AugMixNew4_fold0_tta.pbz2': 1.1194721687089766, 'exp16v6AugmixB3_fold0.pbz2': 0.14839600045579182, 'exp16v6AugmixB4Step2_fold0.pbz2': 1.82365549002064, 'exp16v6AugmixB4Step2_fold0_remove_background.pbz2': 0.6339617162691802, 'exp16v6AugmixB4_fold0.pbz2': 2.7735038813328634, 'exp16v6AugmixB4_fold0_remove_background.pbz2': 2.0210212101657508, 'exp16v6AugmixFullB4_fold0.pbz2': 0.8738583848190067, 'exp16v6sampler_fold0.pbz2': 3.188557229022204, 'exp21b6_fold0.pbz2': 0.5973621085337286, 'exp21b6_fold0_remove_background.pbz2': 1.2507966322681119}. Best is trial 0 with value: 0.8521798853320592.
[I 2022-04-17 07:47:39,071] Trial 1 finished with value: 0.8559862239210065 and parameter

[I 2022-04-17 07:47:54,550] Trial 10 finished with value: 0.8463011626055104 and parameters: {'exp16v4sampler_fold0.pbz2': 0.17871160087958127, 'exp16v6AugMixNew4_fold0.pbz2': 0.014880499804251102, 'exp16v6AugMixNew4_fold0_tta.pbz2': 0.14626434108870012, 'exp16v6AugmixB3_fold0.pbz2': 3.6990204358311716, 'exp16v6AugmixB4Step2_fold0.pbz2': 0.9497801018511124, 'exp16v6AugmixB4Step2_fold0_remove_background.pbz2': 0.19825041040072167, 'exp16v6AugmixB4_fold0.pbz2': 3.956896414343239, 'exp16v6AugmixB4_fold0_remove_background.pbz2': 0.6769156212360656, 'exp16v6AugmixFullB4_fold0.pbz2': 2.6330836944916616, 'exp16v6sampler_fold0.pbz2': 0.8822789948711682, 'exp21b6_fold0.pbz2': 2.928710505099719, 'exp21b6_fold0_remove_background.pbz2': 0.11476746758931111}. Best is trial 2 with value: 0.8635093167701863.
[I 2022-04-17 07:47:56,288] Trial 11 finished with value: 0.8613811912724957 and parameters: {'exp16v4sampler_fold0.pbz2': 2.267242292219125, 'exp16v6AugMixNew4_fold0.pbz2': 3.8743120687799975, '

[I 2022-04-17 07:48:13,883] Trial 21 finished with value: 0.8578794394011785 and parameters: {'exp16v4sampler_fold0.pbz2': 0.5838401395678581, 'exp16v6AugMixNew4_fold0.pbz2': 2.7507159863314437, 'exp16v6AugMixNew4_fold0_tta.pbz2': 1.8238344022757063, 'exp16v6AugmixB3_fold0.pbz2': 1.0126720669735365, 'exp16v6AugmixB4Step2_fold0.pbz2': 2.2192346502316918, 'exp16v6AugmixB4Step2_fold0_remove_background.pbz2': 0.9193023681788239, 'exp16v6AugmixB4_fold0.pbz2': 0.0671968106049956, 'exp16v6AugmixB4_fold0_remove_background.pbz2': 0.40636085625171925, 'exp16v6AugmixFullB4_fold0.pbz2': 0.016149263918445067, 'exp16v6sampler_fold0.pbz2': 1.7512015720878986, 'exp21b6_fold0.pbz2': 1.4669526304401612, 'exp21b6_fold0_remove_background.pbz2': 0.9929234081371587}. Best is trial 19 with value: 0.8649327122153209.
[I 2022-04-17 07:48:15,622] Trial 22 finished with value: 0.8641105271540054 and parameters: {'exp16v4sampler_fold0.pbz2': 0.6492230447642708, 'exp16v6AugMixNew4_fold0.pbz2': 3.391500876563504, '

[I 2022-04-17 07:48:33,100] Trial 32 finished with value: 0.8636546424589903 and parameters: {'exp16v4sampler_fold0.pbz2': 0.3407673662153969, 'exp16v6AugMixNew4_fold0.pbz2': 2.953968190915034, 'exp16v6AugMixNew4_fold0_tta.pbz2': 1.9199134454725246, 'exp16v6AugmixB3_fold0.pbz2': 1.2983424740973346, 'exp16v6AugmixB4Step2_fold0.pbz2': 1.628669099295181, 'exp16v6AugmixB4Step2_fold0_remove_background.pbz2': 0.6859104282363303, 'exp16v6AugmixB4_fold0.pbz2': 0.679336502547828, 'exp16v6AugmixB4_fold0_remove_background.pbz2': 0.3119424240515104, 'exp16v6AugmixFullB4_fold0.pbz2': 1.621233837143361, 'exp16v6sampler_fold0.pbz2': 2.425537831008569, 'exp21b6_fold0.pbz2': 0.907480372392464, 'exp21b6_fold0_remove_background.pbz2': 1.4355893501127541}. Best is trial 19 with value: 0.8649327122153209.
[I 2022-04-17 07:48:34,843] Trial 33 finished with value: 0.8584069915591656 and parameters: {'exp16v4sampler_fold0.pbz2': 0.30074966551689475, 'exp16v6AugMixNew4_fold0.pbz2': 2.155507875852053, 'exp16v6A

[I 2022-04-17 07:48:52,310] Trial 43 finished with value: 0.8675505653766523 and parameters: {'exp16v4sampler_fold0.pbz2': 1.0909614477892304, 'exp16v6AugMixNew4_fold0.pbz2': 3.966578884589196, 'exp16v6AugMixNew4_fold0_tta.pbz2': 3.5688239384689515, 'exp16v6AugmixB3_fold0.pbz2': 0.4446982026837117, 'exp16v6AugmixB4Step2_fold0.pbz2': 0.4796689134543429, 'exp16v6AugmixB4Step2_fold0_remove_background.pbz2': 1.6822627739799478, 'exp16v6AugmixB4_fold0.pbz2': 0.012634383812262208, 'exp16v6AugmixB4_fold0_remove_background.pbz2': 0.3067752083225166, 'exp16v6AugmixFullB4_fold0.pbz2': 3.1974814392662703, 'exp16v6sampler_fold0.pbz2': 3.9705267136530353, 'exp21b6_fold0.pbz2': 1.2389194360396094, 'exp21b6_fold0_remove_background.pbz2': 2.4050549599353666}. Best is trial 43 with value: 0.8675505653766523.
[I 2022-04-17 07:48:54,058] Trial 44 finished with value: 0.8661152253543557 and parameters: {'exp16v4sampler_fold0.pbz2': 1.9450478119746126, 'exp16v6AugMixNew4_fold0.pbz2': 3.901054538107794, 'ex

...

[I 2022-04-17 07:51:44,163] Trial 141 finished with value: 0.8707517120560598 and parameters: {'exp16v4sampler_fold0.pbz2': 0.031918441228517905, 'exp16v6AugMixNew4_fold0.pbz2': 1.5259076650315049, 'exp16v6AugMixNew4_fold0_tta.pbz2': 0.5251492639840339, 'exp16v6AugmixB3_fold0.pbz2': 0.2820520920993817, 'exp16v6AugmixB4Step2_fold0.pbz2': 0.7888521344376733, 'exp16v6AugmixB4Step2_fold0_remove_background.pbz2': 0.0931967349004231, 'exp16v6AugmixB4_fold0.pbz2': 0.4310950227168099, 'exp16v6AugmixB4_fold0_remove_background.pbz2': 0.0009187437168701323, 'exp16v6AugmixFullB4_fold0.pbz2': 2.0687944910416314, 'exp16v6sampler_fold0.pbz2': 1.220202596379448, 'exp21b6_fold0.pbz2': 0.6542294882308916, 'exp21b6_fold0_remove_background.pbz2': 0.46033507791354583}. Best is trial 141 with value: 0.8707517120560598.
[I 2022-04-17 07:51:45,927] Trial 142 finished with value: 0.8698498964803313 and parameters: {'exp16v4sampler_fold0.pbz2': 0.05177914164214539, 'exp16v6AugMixNew4_fold0.pbz2': 1.527800237121

[I 2022-04-17 07:52:01,842] Trial 151 finished with value: 0.8709627329192546 and parameters: {'exp16v4sampler_fold0.pbz2': 0.20392630216423557, 'exp16v6AugMixNew4_fold0.pbz2': 0.908192269283888, 'exp16v6AugMixNew4_fold0_tta.pbz2': 1.0323857061770136, 'exp16v6AugmixB3_fold0.pbz2': 0.15191608255913508, 'exp16v6AugmixB4Step2_fold0.pbz2': 0.8176634063846745, 'exp16v6AugmixB4Step2_fold0_remove_background.pbz2': 0.12505112253661727, 'exp16v6AugmixB4_fold0.pbz2': 0.651517127091944, 'exp16v6AugmixB4_fold0_remove_background.pbz2': 0.07930450297486698, 'exp16v6AugmixFullB4_fold0.pbz2': 2.351757147739716, 'exp16v6sampler_fold0.pbz2': 1.3491473519963801, 'exp21b6_fold0.pbz2': 0.6043477033898164, 'exp21b6_fold0_remove_background.pbz2': 0.2844536712827269}. Best is trial 149 with value: 0.8715360726230292.
[I 2022-04-17 07:52:03,594] Trial 152 finished with value: 0.8665551839464883 and parameters: {'exp16v4sampler_fold0.pbz2': 0.19086230504460183, 'exp16v6AugMixNew4_fold0.pbz2': 0.9480504378133146

[I 2022-04-17 07:52:19,462] Trial 161 finished with value: 0.8706103678929766 and parameters: {'exp16v4sampler_fold0.pbz2': 0.37571835125584463, 'exp16v6AugMixNew4_fold0.pbz2': 0.6289311615792971, 'exp16v6AugMixNew4_fold0_tta.pbz2': 0.6037615110229487, 'exp16v6AugmixB3_fold0.pbz2': 0.03935584443512531, 'exp16v6AugmixB4Step2_fold0.pbz2': 0.8349309262085559, 'exp16v6AugmixB4Step2_fold0_remove_background.pbz2': 0.1755954765703051, 'exp16v6AugmixB4_fold0.pbz2': 0.22335583608422926, 'exp16v6AugmixB4_fold0_remove_background.pbz2': 0.39476550946027006, 'exp16v6AugmixFullB4_fold0.pbz2': 2.5123750443509607, 'exp16v6sampler_fold0.pbz2': 0.7255395073965136, 'exp21b6_fold0.pbz2': 0.5450992502942124, 'exp21b6_fold0_remove_background.pbz2': 0.2672668053688936}. Best is trial 149 with value: 0.8715360726230292.
[I 2022-04-17 07:52:21,217] Trial 162 finished with value: 0.8703316610925307 and parameters: {'exp16v4sampler_fold0.pbz2': 0.3900776359233401, 'exp16v6AugMixNew4_fold0.pbz2': 0.58104728889916

[I 2022-04-17 07:52:37,118] Trial 171 finished with value: 0.8704590699155916 and parameters: {'exp16v4sampler_fold0.pbz2': 0.21118573849709227, 'exp16v6AugMixNew4_fold0.pbz2': 0.37397717649257395, 'exp16v6AugMixNew4_fold0_tta.pbz2': 0.7295318229486052, 'exp16v6AugmixB3_fold0.pbz2': 0.21617649446279225, 'exp16v6AugmixB4Step2_fold0.pbz2': 0.5124432997267303, 'exp16v6AugmixB4Step2_fold0_remove_background.pbz2': 0.10167887501179154, 'exp16v6AugmixB4_fold0.pbz2': 0.11523501476193218, 'exp16v6AugmixB4_fold0_remove_background.pbz2': 0.5586256992745866, 'exp16v6AugmixFullB4_fold0.pbz2': 2.296939288403612, 'exp16v6sampler_fold0.pbz2': 0.5899279332591162, 'exp21b6_fold0.pbz2': 0.8221916378477853, 'exp21b6_fold0_remove_background.pbz2': 0.096813968431502}. Best is trial 149 with value: 0.8715360726230292.


CPU times: user 5min 1s, sys: 984 ms, total: 5min 2s
Wall time: 5min 1s


In [21]:
study.trials_dataframe().sort_values("value")

,number,value,datetime_start,datetime_complete,duration,params_exp16v4sampler_fold0.pbz2,params_exp16v6AugMixNew4_fold0.pbz2,params_exp16v6AugMixNew4_fold0_tta.pbz2,params_exp16v6AugmixB3_fold0.pbz2,params_exp16v6AugmixB4Step2_fold0.pbz2,params_exp16v6AugmixB4Step2_fold0_remove_background.pbz2,params_exp16v6AugmixB4_fold0.pbz2,params_exp16v6AugmixB4_fold0_remove_background.pbz2,params_exp16v6AugmixFullB4_fold0.pbz2,params_exp16v6sampler_fold0.pbz2,params_exp21b6_fold0.pbz2,params_exp21b6_fold0_remove_background.pbz2,state
10,10,0.846301,2022-04-17 07:47:52.815587,2022-04-17 07:47:54.549774,0 days 00:00:01.734187,0.178712,0.014880,0.146264,3.699020,0.949780,0.198250,3.956896,0.676916,2.633084,0.882279,2.928711,0.114767,COMPLETE
0,0,0.852180,2022-04-17 07:47:35.621763,2022-04-17 07:47:37.356027,0 days 00:00:01.734264,1.096957,1.461501,1.119472,0.148396,1.823655,0.633962,2.773504,2.021021,0.873858,3.188557,0.597362,1.250797,COMPLETE
28,28,0.852425,2022-04-17 07:48:24.349086,2022-04-17 07:48:26.094618,0 days 00:00:01.745532,0.395236,0.831723,2.181878,0.543763,0.018327,1.201297,1.078133,0.018134,0.464293,2.808444,0.964699,3.964384,COMPLETE
5,5,0.852443,2022-04-17 07:47:44.208943,2022-04-17 07:47:45.922115,0 days 00:00:01.713172,3.311079,1.034762,1.007801,0.246492,0.284183,3.844894,2.402934,2.944226,2.170480,1.586686,1.149798,3.452536,COMPLETE
9,9,0.852783,2022-04-17 07:47:51.096287,2022-04-17 07:47:52.814024,0 days 00:00:01.717737,2.221246,1.525933,2.036819,2.111759,2.709646,1.370647,3.101525,1.889178,1.453604,0.018290,3.882650,2.487844,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,159,0.871160,2022-04-17 07:52:14.191772,2022-04-17 07:52:15.944801,0 days 00:00:01.753029,0.391691,0.584336,0.572418,0.008212,0.827662,0.009719,0.113986,0.378704,2.474418,0.747524,0.806701,0.248627,COMPLETE
155,155,0.871291,2022-04-17 07:52:07.134260,2022-04-17 07:52:08.891950,0 days 00:00:01.757690,0.089406,0.755778,0.608788,0.045826,0.827735,0.000257,0.312292,0.305556,2.492926,0.731894,0.565480,0.580005,COMPLETE
167,167,0.871331,2022-04-17 07:52:28.306437,2022-04-17 07:52:30.069739,0 days 00:00:01.763302,0.417624,0.371641,0.913456,0.025346,0.647252,0.002049,0.140800,0.416886,2.296501,0.801865,0.710185,0.241254,COMPLETE
154,154,0.871419,2022-04-17 07:52:05.377091,2022-04-17 07:52:07.132565,0 days 00:00:01.755474,0.077503,1.288827,0.685451,0.018527,0.824987,0.065729,0.307463,0.255140,2.471063,0.613262,0.595144,0.239974,COMPLETE


In [22]:
study.best_params

{'exp16v4sampler_fold0.pbz2': 0.18324696342748883,
 'exp16v6AugMixNew4_fold0.pbz2': 0.9401242469371447,
 'exp16v6AugMixNew4_fold0_tta.pbz2': 0.6644063537736292,
 'exp16v6AugmixB3_fold0.pbz2': 0.1626001970228471,
 'exp16v6AugmixB4Step2_fold0.pbz2': 0.6921214779982734,
 'exp16v6AugmixB4Step2_fold0_remove_background.pbz2': 0.10843876264566758,
 'exp16v6AugmixB4_fold0.pbz2': 0.31072842462504724,
 'exp16v6AugmixB4_fold0_remove_background.pbz2': 0.08345941225482527,
 'exp16v6AugmixFullB4_fold0.pbz2': 2.392914485644236,
 'exp16v6sampler_fold0.pbz2': 0.7085560248541085,
 'exp21b6_fold0.pbz2': 0.8070710457493987,
 'exp21b6_fold0_remove_background.pbz2': 0.28835546888492014}

In [23]:
best_weights = list(study.best_params.values())

In [24]:
# weighted result
get_score([1] + best_weights)

0.8715360726230292

In [25]:
# un-weighted result
get_score([1] * len(exps))

0.8566431756649148